In [1]:
import re
import pandas as pd

from IPython.display import display

In [2]:
df = pd.read_csv("lpdp_cs.csv")
df_qs = pd.read_csv("qs_2022_simple.csv")
df_cwur = pd.read_csv("cwur_2021-2022_simple.csv")
df_the = pd.read_csv("the_2022_simple.csv")
dfs = [df, df_qs, df_cwur, df_the]
# Remove "Reporter"
df_the = df_the[df_the["THE Rank 2022"].ne("Reporter")]
# Remove leading/trailing spaces
for _df in dfs:
    _df["Name"] = _df["Name"].str.strip()
# Convert
df_cwur["CWUR 2021-2022 Rank"] = df_cwur["CWUR 2021-2022 Rank"].astype(int).astype(str)

In [3]:
# Replace similar name
def replace_similar_name(df_ref, df_proc):
    conflicted_i = set()
    for k, name in df_ref["Name"].items():
        tmp1 = df_proc["Name"].str.contains(name, regex=False)
        q = re.sub("[\(\[].*?[\)\]]", "", name).strip()
        tmp2 = df_proc["Name"].str.contains(q, regex=False)
        tmp = (tmp1 | tmp2)
        if tmp.sum() > 1:
            matched_i = set(df_proc["Name"][tmp].index.values)
            if matched_i.intersection(conflicted_i):
                continue
            conflicted_i.update(matched_i)
            i = df_proc["Name"][tmp].apply(lambda x: abs(len(x)-len(name))).idxmin()
        else:
            i = tmp.idxmax()
        if tmp.any():
            df_proc.iloc[i, 1] = name
replace_similar_name(df, df_the)
replace_similar_name(df, df_cwur)

/tmp/ipykernel_12337/3695666292.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_proc.iloc[i, 1] = name


In [4]:
df_merged = df.merge(df_qs, how="left", on="Name").merge(df_cwur, how="left", on="Name").merge(df_the, how="left", on="Name")
df_merged.to_csv("cs_lpdp_qs_the_cwur.csv", index=False)
df_merged

,No,Name,Negara,QS 2022,CWUR 2021-2022 Rank,THE Rank 2022
0,1,University of Oxford,Inggris,2,5,1
1,2,Harvard University,Amerika Serikat,5,1,2
2,3,Stanford University,Amerika Serikat,3,3,4
3,4,Massachusetts Institute of Technology (MIT),Amerika Serikat,1,2,5
4,5,University of Cambridge,Inggris,3,4,5
...,...,...,...,...,...,...
153,154,Uppsala University,Swedia,124,88,131
154,155,National Tsing Hua University,Taiwan,180,382,351–400
155,156,National Yang Ming Chiao Tung University,Taiwan,268,NaN,301–350
156,157,Taipei Medical University,Taiwan,NaN,712,201–250


In [5]:
# df_merged[df_merged.isna().any(axis=1)]